# **附录4：关于神经网络 2**

在 1 的基础上，补充记录 **交叉熵（Cross-Entropy）** 损失函数、**Softmax** 输出层激活函数、损失函数的 **正则化（Regularization）** 、权重参数 w 的初始值设定、**超参数（hyper-parameter）** 的选择等内容，依然是基于 **[Neural Networks and Deep Learning 教程](http://neuralnetworksanddeeplearning.com/chap3.html)** 。

> The philosophy is that the best entree to the plethora of available techniques is in-depth study of a few of the most important.（本教程依据的哲学是：入门当下众多技术的最好方式是深入学习其中最重要的那些。）


&emsp;

## **损失函数——交叉熵（Cross-Entropy）**

在 1 中，输出层神经元使用的是 **sigmoid 激活函数**：

$$a=\sigma(z)=\frac{1}{1+e^{-z}}$$

搭配 **二次损失函数（Quadratic Cost Function）**：

$$C = \frac{(y-a)^2}{2}$$

这两者的搭配使用会造成一个问题：学习速率慢，即参数 w 和 b 相对损失函数的变化、变化很慢

$$\frac{\partial C}{\partial w} = (a-y)\sigma'(z)x$$

$$\frac{\partial C}{\partial b} = (a-y)\sigma'(z)$$

这主要是和偏导数中的 $\sigma'(z)$ 有关（当 $\sigma(z)$ 趋近于 1 时，曲线平缓，即导数值 $\sigma '(z)$ 趋近于 0）， 所以引入了 **交叉熵损失函数**，后面会看到，它与 sigmoid 激活函数的搭配使用，会回避掉 $\sigma'(z)$

$$C = - \frac{1}{n} \sum \limits_x [y \ln a + (1-y) \ln(1-a)]$$

$$\frac{\partial C}{\partial w_j} = \frac{1}{n} \sum \limits_x x_j (\sigma(z)-y)$$

$$\frac{\partial C}{\partial b} = \frac{1}{n} \sum \limits_x (\sigma (z) - y)$$

由上面偏导数公式可以看出，使用交叉熵损失函数后，**学习速率的变化主要取决于输出值与预期输出值间的偏差，偏差（即 $\sigma(z)-y$）越大，学习速率越快**

&emsp;

## **输出层激活函数——Softmax**


$$a^L_j = \frac{e^{z^L_j}}{\sum_k e^{z^L_k}}$$


输出层神经元采用 Softmax 作为激活函数后，所有输出结果构成一个概率分布，总和为 1。对比 sigmoid 激活函数，softmax 有两个特点：单调性，输入值 $z^L_j$